# 将JD数据转换为sharegpt格式，支持在lf上训练

In [3]:
data_path = '/braincoder-extreme-nas/datasets/Jedi/datasets/'
import os
import glob
import json
import re

# 递归查找所有jsonl文件
jsonl_files = []
for root, dirs, files in os.walk(data_path):
    for file in files:
        if file.endswith('.jsonl'):
            jsonl_files.append(os.path.join(root, file))

print(jsonl_files)
print(len(jsonl_files))


system_values = set()
# 正则表达式匹配整行（包括前后的*和换行符）
start_set = set()
data_demo = []
system_num = {}

for jsonl_file in jsonl_files:
    with open(jsonl_file, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line.strip())
                if 'conversations' in data:
                    for conv in data['conversations']:
                        if conv.get('from') == 'system' :
                            query = conv.get('value', '')
                            if query[:20] in start_set:
                                system_num[query[:20]] = system_num.get(query[:20], 0) + 1
                                continue

                            system_values.add(query)
                            data_demo.append(data)
                            start_set.add(query[:20])
                            continue
            except json.JSONDecodeError:
                continue

print(f"Total unique system values: {len(system_values)}")
print(f"Total data demo: {len(data_demo)}")
print(system_num.values())
print("\nUnique system values:")



['/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/android_control-v2/android_control-v2.jsonl', '/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/coat-v2/coat-v2.jsonl', '/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/docvqa_grounding/docvqa_grounding.jsonl', '/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/guiact-web-multi/guiact-web-multi.jsonl', '/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/guiact-web-single-v2/guiact-web-single-v2.jsonl', '/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/guide_si_10k-v2/guide_si_10k-v2.jsonl', '/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/guienv/guienv.jsonl', '/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/mind2web_train_v1.0.1/mind2web_train_v1.0.1.jsonl', '/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/omniact/omniact.jsonl', '/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/osatlas_ui_tars_cleaned/osatlas_ui_tars_cleaned.jsonl', '/braincoder-extreme-nas/datase

In [ ]:
# 将system values保存到文件中
output_file = 'system_values.txt'
with open(output_file, 'w', encoding='utf-8') as f:
    for value in sorted(system_values):
        f.write(f"{value}\n")
        f.write("*"*100+"\n")
with open('data_demo.jsonl', 'w', encoding='utf-8') as f:
    for data in data_demo:
        f.write(json.dumps(data, ensure_ascii=False) + "\n")
print(f"System values have been saved to {output_file}")

with open('start_set.txt', 'r', encoding='utf-8') as f:
    start_list = list(start_set)
    

with open('system_num.jsonl', 'w', encoding='utf-8') as f:
    for value in sorted(system_num):
        f.write(f"{value}\n")
        f.write("*"*100+"\n")


NameError: name 'system_values' is not defined

In [15]:
text = """You are a helpful assistant.

# Tools

You may call one or more functions to assist with the user query.

You are provided with function signatures within <tools></tools> XML tags:
<tools>
{"type": "function", "function": {"name": "computer_use", "description": "Use a mouse and keyboard to interact with a computer, and take screenshots.\n* This is an interface to a desktop GUI. You do not have access to a terminal or applications menu. You must click on desktop icons to start applications.\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions. E.g. if you click on Firefox and a window doesn't open, try wait and taking another screenshot.\n* The screen's resolution is {{screen_width}}x{{screen_height}}.\n* Whenever you intend to move the cursor to click on an element like an icon, you should consult a screenshot to determine the coordinates of the element before moving the cursor.\n* If you tried clicking on a program or link but it failed to load, even after waiting, try adjusting your cursor position so that the tip of the cursor visually falls on the element that you want to click.\n* Make sure to click any buttons, links, icons, etc with the cursor tip in the center of the element. Don't click boxes on their edges unless asked.", "parameters": {"properties": {"action": {"description": "The action to perform. The available actions are:\n* `key`: Performs key down presses on the arguments passed in order, then performs key releases in reverse order.\n* `type`: Type a string of text on the keyboard.\n* `mouse_move`: Move the cursor to a specified (x, y) pixel coordinate on the screen.\n* `left_click`: Click the left mouse button.\n* `left_click_drag`: Click and drag the cursor to a specified (x, y) pixel coordinate on the screen.\n* `right_click`: Click the right mouse button.\n* `middle_click`: Click the middle mouse button.\n* `double_click`: Double-click the left mouse button.\n* `scroll`: Performs a scroll of the mouse scroll wheel.\n* `wait`: Wait specified seconds for the change to happen.\n* `terminate`: Terminate the current task and report its completion status.", "enum": ["key", "type", "mouse_move", "left_click", "left_click_drag", "right_click", "middle_click", "double_click", "scroll", "wait", "terminate"], "type": "string"}, "keys": {"description": "Required only by `action=key`.", "type": "array"}, "text": {"description": "Required only by `action=type`.", "type": "string"}, "coordinate": {"description": "(x, y): The x (pixels from the left edge) and y (pixels from the top edge) coordinates to move the mouse to. Required only by `action=mouse_move` and `action=left_click_drag`.", "type": "array"}, "pixels": {"description": "The amount of scrolling to perform. Positive values scroll up, negative values scroll down. Required only by `action=scroll`.", "type": "number"}, "time": {"description": "The seconds to wait. Required only by `action=wait`.", "type": "number"}, "status": {"description": "The status of the task. Required only by `action=terminate`.", "type": "string", "enum": ["success", "failure"]}}, "required": ["action"], "type": "object"}}}
</tools>

For each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:
<tool_call>
{"name": <function-name>, "arguments": <args-json-object>}
</tool_call>"""

escaped_text = repr(text) # 去掉首尾的引号
print(escaped_text)

'You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "computer_use", "description": "Use a mouse and keyboard to interact with a computer, and take screenshots.\n* This is an interface to a desktop GUI. You do not have access to a terminal or applications menu. You must click on desktop icons to start applications.\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions. E.g. if you click on Firefox and a window doesn\'t open, try wait and taking another screenshot.\n* The screen\'s resolution is {{screen_width}}x{{screen_height}}.\n* Whenever you intend to move the cursor to click on an element like an icon, you should consult a screenshot to determine the coordinates of the element before moving the cur

In [10]:
data_path = '/braincoder-extreme-nas/datasets/Jedi/images'
import os

import os

def get_last_level_dirs(parent_dir):
    last_level_dirs = []
    for root, dirs, files in os.walk(parent_dir):
        if not dirs:  # 当前目录没有子目录时
            last_level_dirs.append(root)
    return last_level_dirs

# 示例
target_dir = "/path/to/your/folder"
result = get_last_level_dirs(data_path)
print("最后一级目录列表:", result)




最后一级目录列表: ['/braincoder-extreme-nas/datasets/Jedi/images/component_library_snap_icon_data', '/braincoder-extreme-nas/datasets/Jedi/images/component_v1_130k', '/braincoder-extreme-nas/datasets/Jedi/images/doc_images', '/braincoder-extreme-nas/datasets/Jedi/images/doc_scroll', '/braincoder-extreme-nas/datasets/Jedi/images/ethercalc_data', '/braincoder-extreme-nas/datasets/Jedi/images/figma400k', '/braincoder-extreme-nas/datasets/Jedi/images/final_1.5m', '/braincoder-extreme-nas/datasets/Jedi/images/icons_v0122/images_grounded', '/braincoder-extreme-nas/datasets/Jedi/images/icons_v0122/images_pure_color_background/FindMyAssets', '/braincoder-extreme-nas/datasets/Jedi/images/icons_v0122/images_pure_color_background/LibreOffice', '/braincoder-extreme-nas/datasets/Jedi/images/icons_v0122/images_pure_color_background/MicrosoftSolitaireImages', '/braincoder-extreme-nas/datasets/Jedi/images/icons_v0122/images_pure_color_background/PowerpointAssets', '/braincoder-extreme-nas/datasets/Jedi/images

In [11]:
import os
import glob
import zipfile
import tarfile
import shutil

def find_and_merge_compressed_files(directory):
    # 查找所有可能的压缩文件分片
    zip_parts = glob.glob(os.path.join(directory, '*.zip*'))
    tar_parts = glob.glob(os.path.join(directory, '*.tar*'))
    
    # 处理zip分片
    if zip_parts:
        print(f"找到 {len(zip_parts)} 个可能的zip分片文件")
        
        # 按文件名排序，确保正确的合并顺序
        zip_parts.sort()
        
        # 确定合并后的文件名
        base_name = os.path.basename(zip_parts[0])
        if '.zip' in base_name:
            merged_name = base_name.split('.zip')[0] + '.zip'
        else:
            merged_name = 'merged.zip'
        
        merged_path = os.path.join(directory, merged_name)
        
        # 合并文件
        print(f"正在合并文件到 {merged_name}...")
        with open(merged_path, 'wb') as merged_file:
            for part in zip_parts:
                with open(part, 'rb') as part_file:
                    shutil.copyfileobj(part_file, merged_file)
        
        # 尝试解压
        print("尝试解压合并后的zip文件...")
        try:
            with zipfile.ZipFile(merged_path, 'r') as zip_ref:
                zip_ref.extractall(directory)
            print("解压成功！")
        except zipfile.BadZipFile:
            print("合并后的文件不是有效的zip文件，可能合并失败")
    
    # 处理tar分片
    elif tar_parts:
        print(f"找到 {len(tar_parts)} 个可能的tar分片文件")
        
        # 按文件名排序
        tar_parts.sort()
        
        # 确定合并后的文件名
        base_name = os.path.basename(tar_parts[0])
        if '.tar' in base_name:
            merged_name = base_name.split('.tar')[0] + '.tar'
        else:
            merged_name = 'merged.tar'
        
        merged_path = os.path.join(directory, merged_name)
        
        # 合并文件
        print(f"正在合并文件到 {merged_name}...")
        with open(merged_path, 'wb') as merged_file:
            for part in tar_parts:
                with open(part, 'rb') as part_file:
                    shutil.copyfileobj(part_file, merged_file)
        
        # 尝试解压
        print("尝试解压合并后的tar文件...")
        try:
            if merged_name.endswith('.tar.gz') or merged_name.endswith('.tgz'):
                with tarfile.open(merged_path, 'r:gz') as tar_ref:
                    tar_ref.extractall(directory)
            elif merged_name.endswith('.tar.bz2') or merged_name.endswith('.tbz'):
                with tarfile.open(merged_path, 'r:bz2') as tar_ref:
                    tar_ref.extractall(directory)
            else:
                with tarfile.open(merged_path, 'r') as tar_ref:
                    tar_ref.extractall(directory)
            print("解压成功！")
        except tarfile.TarError:
            print("合并后的文件不是有效的tar文件，可能合并失败")
    
    else:
        print("没有找到任何压缩文件分片")
        
if __name__ == "__main__":
    target_directory = "/braincoder-extreme-nas/datasets/Jedi/images/component_library_snap_icon_data"
    find_and_merge_compressed_files(target_directory)

找到 2 个可能的tar分片文件
正在合并文件到 component_library_snap_icon_data.tar...
尝试解压合并后的tar文件...


KeyboardInterrupt: 

In [ ]:
jedi_data_path = '/braincoder-extreme-nas/datasets/Jedi/datasets'
ac_data_path = '/braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/android_control-v2/android_control-v2.jsonl'
img_paht = '/braincoder-extreme-nas/datasets/aguvis-stage1/data/aguvis/images' if 'aguvis++' in ac_data_path else '/braincoder-extreme-nas/datasets/Jedi/images'
import os
import json

ac_data =[ json.loads(line) for line in open(ac_data_path, 'r')]
img_path_list = [item['image'] for item in ac_data]
print(len(ac_data))
print(ac_data[0])

54678
{'image': '30/screenshot_0.png', 'conversations': [{'from': 'system', 'value': 'You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "mobile_use", "description": "Use a touchscreen to interact with a mobile device, and take screenshots.\\n* This is an interface to a mobile device with touchscreen. You can perform actions like clicking, typing, swiping, etc.\\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions.\\n* The screen\'s resolution is 952x2156.\\n* Make sure to click any buttons, links, icons, etc with the cursor tip in the center of the element. Don\'t click boxes on their edges unless asked.", "parameters": {"properties": {"action": {"description": "The action to perform. The available a

In [22]:
jedi_data_path = '/braincoder-extreme-nas/datasets/Jedi/'
import yaml
import os

# 读取 YAML 文件
with open(os.path.join(jedi_data_path, 'datasets.yaml'), 'r') as file:
    data = yaml.safe_load(file)

datasets_list = data['datasets']
print(datasets_list)

def read_jsonl(jsonl_path):
    with open(jsonl_path, 'r') as file:
        return [json.loads(line) for line in file]
# datasets_list = [{'images_folder': 'osatlas', 'jsonl_path': 'datasets/aguvis++/osatlas_ui_tars_cleaned/osatlas_ui_tars_cleaned.jsonl', 'sampling_strategy': 'all'}]
for tmp_dataset in datasets_list:
    if 'aguvis' in tmp_dataset['jsonl_path']:
        continue
    images_folder = tmp_dataset['images_folder']
    jsonl_path = os.path.join(jedi_data_path, tmp_dataset['jsonl_path'])
    _dataset = read_jsonl(jsonl_path)
    for item in _dataset:
        image_path = item['image']
        absolute_path = os.path.join(jedi_data_path, 'images', images_folder, image_path)
        #确定长宽可以被28*28整除
        image = Image.open(absolute_path)
        width, height = image.size
        if width % 28 != 0 or height % 28 != 0:
            print(f"Image size not divisible by 28: {width}x{height}")
            break
        if not os.path.exists(absolute_path):
            print(f"Image not found: {absolute_path}")
            break
        print(absolute_path)
        break




[{'images_folder': 'seeclick_web_imgs', 'jsonl_path': 'datasets/aguvis++/seeclick_ui_tars_cleaned_fixed/seeclick_ui_tars_cleaned_fixed.jsonl', 'sampling_strategy': 'all'}, {'images_folder': 'guienv', 'jsonl_path': 'datasets/aguvis++/guienv/guienv.jsonl', 'sampling_strategy': 'all'}, {'images_folder': 'webui350k', 'jsonl_path': 'datasets/aguvis++/webui350k/webui350k.jsonl', 'sampling_strategy': 'all'}, {'images_folder': 'omniact', 'jsonl_path': 'datasets/aguvis++/omniact/omniact.jsonl', 'sampling_strategy': 'all'}, {'images_folder': 'seeclick_web_imgs', 'jsonl_path': 'datasets/aguvis++/seeclick_mi_ui_tars_cleaned/seeclick_mi_ui_tars_cleaned.jsonl', 'sampling_strategy': 'all'}, {'images_folder': 'osatlas', 'jsonl_path': 'datasets/aguvis++/osatlas_ui_tars_cleaned/osatlas_ui_tars_cleaned.jsonl', 'sampling_strategy': 'all'}, {'images_folder': 'widget_captioning', 'jsonl_path': 'datasets/aguvis++/widget_captioning/widget_captioning.jsonl', 'sampling_strategy': 'all'}, {'images_folder': 'rico

FileNotFoundError: [Errno 2] No such file or directory: '/braincoder-extreme-nas/datasets/Jedi/images/osatlas/windows_20240823_072238_before_screenshot_sub1_6_6.png'

In [9]:
def chage_data_sharegpt(item):
    item['messages'] = []
    for conversation in item['conversations']:
        item['messages'].append({
                'content': conversation['value'],
                'role': conversation['from']
            })
    if 'conversations' in item:
        del item['conversations']
    if 'image' in item:
        del item['image']
    if 'image_id' in item:
        del item['image_id']
    return item

item = {
  "image": "2ef30220f266823a04ac1c608173efb9.png",
  "conversations": [
    {
      "from": "system",
      "value": "You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{\"type\": \"function\", \"function\": {\"name\": \"computer_use\", \"description\": \"Use a mouse and keyboard to interact with a computer, and take screenshots.\\n* This is an interface to a desktop GUI. You do not have access to a terminal or applications menu. You must click on desktop icons to start applications.\\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions. E.g. if you click on Firefox and a window doesn't open, try wait and taking another screenshot.\\n* The screen's resolution is 1932x1092.\\n* Whenever you intend to move the cursor to click on an element like an icon, you should consult a screenshot to determine the coordinates of the element before moving the cursor.\\n* If you tried clicking on a program or link but it failed to load, even after waiting, try adjusting your cursor position so that the tip of the cursor visually falls on the element that you want to click.\\n* Make sure to click any buttons, links, icons, etc with the cursor tip in the center of the element. Don't click boxes on their edges unless asked.\", \"parameters\": {\"properties\": {\"action\": {\"description\": \"The action to perform. The available actions are:\\n* `key`: Performs key down presses on the arguments passed in order, then performs key releases in reverse order.\\n* `type`: Type a string of text on the keyboard.\\n* `mouse_move`: Move the cursor to a specified (x, y) pixel coordinate on the screen.\\n* `left_click`: Click the left mouse button.\\n* `left_click_drag`: Click and drag the cursor to a specified (x, y) pixel coordinate on the screen.\\n* `right_click`: Click the right mouse button.\\n* `middle_click`: Click the middle mouse button.\\n* `double_click`: Double-click the left mouse button.\\n* `scroll`: Performs a scroll of the mouse scroll wheel.\\n* `wait`: Wait specified seconds for the change to happen.\\n* `terminate`: Terminate the current task and report its completion status.\", \"enum\": [\"key\", \"type\", \"mouse_move\", \"left_click\", \"left_click_drag\", \"right_click\", \"middle_click\", \"double_click\", \"scroll\", \"wait\", \"terminate\"], \"type\": \"string\"}, \"keys\": {\"description\": \"Required only by `action=key`.\", \"type\": \"array\"}, \"text\": {\"description\": \"Required only by `action=type`.\", \"type\": \"string\"}, \"coordinate\": {\"description\": \"(x, y): The x (pixels from the left edge) and y (pixels from the top edge) coordinates to move the mouse to. Required only by `action=mouse_move` and `action=left_click_drag`.\", \"type\": \"array\"}, \"pixels\": {\"description\": \"The amount of scrolling to perform. Positive values scroll up, negative values scroll down. Required only by `action=scroll`.\", \"type\": \"number\"}, \"time\": {\"description\": \"The seconds to wait. Required only by `action=wait`.\", \"type\": \"number\"}, \"status\": {\"description\": \"The status of the task. Required only by `action=terminate`.\", \"type\": \"string\", \"enum\": [\"success\", \"failure\"]}}, \"required\": [\"action\"], \"type\": \"object\"}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call>"
    },
    {
      "from": "human",
      "value": "<image>\nRight-click on 'Back to Top'"
    },
    {
      "from": "gpt",
      "value": "<tool_call>\n{\"name\":\"computer_use\",\"arguments\":{\"action\":\"right_click\",\"coordinate\":[59,854]}}\n</tool_call>"
    }
  ],
  "image_id": "2ef30220f266823a04ac1c608173efb9.png",
  "images": [
    "/primus_datasets/datasets/Jedi_LF/datasets/aguvis++/seeclick_mi_ui_tars_cleaned/images/2ef30220f266823a04ac1c608173efb9.png"
  ]
}
new_data = chage_data_sharegpt(item)
import json
print(json.dumps(new_data, indent=4))

{
    "images": [
        "/primus_datasets/datasets/Jedi_LF/datasets/aguvis++/seeclick_mi_ui_tars_cleaned/images/2ef30220f266823a04ac1c608173efb9.png"
    ],
    "messages": [
        {
            "content": "You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{\"type\": \"function\", \"function\": {\"name\": \"computer_use\", \"description\": \"Use a mouse and keyboard to interact with a computer, and take screenshots.\\n* This is an interface to a desktop GUI. You do not have access to a terminal or applications menu. You must click on desktop icons to start applications.\\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions. E.g. if you click on Firefox and a window doesn't open, try wait and taking another screenshot.\\n* The screen's re

In [1]:
data_path = '/braincoder-extreme-nas/datasets/GUI-Net-1M/merged_files/training_data/gui_video_full/0000099e-cd6e-46ce-96b1-1057ae66104e.png'
image = Image.open(data_path)
width, height = image.size
if width % 28 != 0 or height % 28 != 0:
    print(f"Image size not divisible by 28: {width}x{height}")

NameError: name 'Image' is not defined

In [12]:
from transformers.models.qwen2_vl.image_processing_qwen2_vl_fast import smart_resize
from PIL import Image
import os
def resize_image(image):
# 计算调整后的尺寸
    assert isinstance(image, str) and os.path.exists(image), f"Invalid input image path: {image}"
    image_path = image
    assert os.path.exists(image_path) and os.path.isfile(image_path), f"Invalid input image path: {image_path}"
    image = Image.open(image_path).convert('RGB')
    assert isinstance(image, Image.Image), "Invalid input image."
    resized_height, resized_width = smart_resize(
        image.height,
        image.width,
        factor=14,
        min_pixels=4 * 14 * 14,
        max_pixels=16384 * 14 * 14,)
    # 图片resize
    resized_image = image.resize((resized_width, resized_height))
    return resized_image, resized_height, resized_width

resized_image, resized_height, resized_width = resize_image('/braincoder-extreme-nas/datasets/Jedi/images/icons_v0122/images_grounded/21563.png')
print(resized_height, resized_width)


In [28]:
def exec_box_from_gpt(item, resized_width_factor, resized_height_factor):
    new_conversations = []
    for conversation in item['conversations']:
        if conversation['from'] == 'gpt' and 'value' in conversation:
            input_str = conversation['value']
            
            # 改进后的正则表达式（支持空格/逗号分隔）
            coord_match = re.search(
                r'(?:bounding\s+box:?\s*)?'
                r'[$$$]?\s*'
                r'([-+]?\d*\.\d+|[-+]?\d+)'
                r'(?:\s*[, ]\s*)'
                r'([-+]?\d*\.\d+|[-+]?\d+)'
                r'(?:\s*[, ]\s*)'
                r'([-+]?\d*\.\d+|[-+]?\d+)'
                r'(?:\s*[, ]\s*)'
                r'([-+]?\d*\.\d+|[-+]?\d+)'
                r'[$$$]?',
                input_str,
                re.VERBOSE
            )
            
            if coord_match:
                try:
                    # 提取并缩放坐标
                    x, y, w, h = map(float, coord_match.groups())
                    scaled_x = x * resized_width_factor
                    scaled_y = y * resized_height_factor
                    scaled_w = w * resized_width_factor
                    scaled_h = h * resized_height_factor
                    
                    # 确定原始格式的分隔符
                    separator = ',' if ',' in coord_match.group(0) else ' '
                    
                    # 重建坐标字符串（保持原始格式）
                    if '(' in coord_match.group(0):
                        new_coord = f"({scaled_x}{separator}{scaled_y}{separator}{scaled_w}{separator}{scaled_h})"
                    elif '[' in coord_match.group(0):
                        new_coord = f"[{scaled_x}{separator}{scaled_y}{separator}{scaled_w}{separator}{scaled_h}]"
                    else:
                        new_coord = f"{scaled_x}{separator}{scaled_y}{separator}{scaled_w}{separator}{scaled_h}"
                    
                    # 保留原始文本结构
                    prefix = input_str[:coord_match.start()]
                    suffix = input_str[coord_match.end():]
                    
                    new_conversation = conversation.copy()
                    new_conversation['value'] = prefix + new_coord + suffix
                    new_conversations.append(new_conversation)
                    continue
                
                except (ValueError, IndexError) as e:
                    print(f"Error processing coordinates: {e}")
        
        # 非GPT消息或处理失败时保留原样
        new_conversations.append(conversation)
    
    item['conversations'] = new_conversations
    return item
# /braincoder-extreme-nas/datasets/Jedi/datasets/aguvis++/osatlas_ui_tars_cleaned/osatlas_ui_tars_cleaned.jsonl
data ={
	"image": "30/screenshot_0.png",
	"conversations": [{
		"from": "system",
		"value": "You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{\"type\": \"function\", \"function\": {\"name\": \"mobile_use\", \"description\": \"Use a touchscreen to interact with a mobile device, and take screenshots.\\n* This is an interface to a mobile device with touchscreen. You can perform actions like clicking, typing, swiping, etc.\\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions.\\n* The screen's resolution is 952x2156.\\n* Make sure to click any buttons, links, icons, etc with the cursor tip in the center of the element. Don't click boxes on their edges unless asked.\", \"parameters\": {\"properties\": {\"action\": {\"description\": \"The action to perform. The available actions are:\\n* `key`: Perform a key event on the mobile device.\\n    - This supports adb's `keyevent` syntax.\\n    - Examples: \\\"volume_up\\\", \\\"volume_down\\\", \\\"power\\\", \\\"camera\\\", \\\"clear\\\".\\n* `click`: Click the point on the screen with coordinate (x, y).\\n* `long_press`: Press the point on the screen with coordinate (x, y) for specified seconds.\\n* `swipe`: Swipe from the starting point with coordinate (x, y) to the end point with coordinates2 (x2, y2).\\n* `type`: Input the specified text into the activated input box.\\n* `system_button`: Press the system button.\\n* `open`: Open an app on the device.\\n* `wait`: Wait specified seconds for the change to happen.\\n* `terminate`: Terminate the current task and report its completion status.\", \"enum\": [\"key\", \"click\", \"long_press\", \"swipe\", \"type\", \"system_button\", \"open\", \"wait\", \"terminate\"], \"type\": \"string\"}, \"coordinate\": {\"description\": \"(x, y): The x (pixels from the left edge) and y (pixels from the top edge) coordinates to move the mouse to. Required only by `action=click`, `action=long_press`, and `action=swipe`.\", \"type\": \"array\"}, \"coordinate2\": {\"description\": \"(x, y): The x (pixels from the left edge) and y (pixels from the top edge) coordinates to move the mouse to. Required only by `action=swipe`.\", \"type\": \"array\"}, \"text\": {\"description\": \"Required only by `action=key`, `action=type`, and `action=open`.\", \"type\": \"string\"}, \"time\": {\"description\": \"The seconds to wait. Required only by `action=long_press` and `action=wait`.\", \"type\": \"number\"}, \"button\": {\"description\": \"Back means returning to the previous interface, Home means returning to the desktop, Menu means opening the application background menu, and Enter means pressing the enter. Required only by `action=system_button`\", \"enum\": [\"Back\", \"Home\", \"Menu\", \"Enter\"], \"type\": \"string\"}, \"status\": {\"description\": \"The status of the task. Required only by `action=terminate`.\", \"type\": \"string\", \"enum\": [\"success\", \"failure\"]}}, \"required\": [\"action\"], \"type\": \"object\"}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call>"
	}, {
		"from": "human",
		"value": "<image>\nPlease generate the next move according to the UI screenshot, instruction and previous actions.\n\nInstruction: Compare the prices of the Shoes similar to men's athletic shoes in the Etsy app\n\nPrevious actions:\nNone"
	}, {
		"from": "gpt",
		"value": "Action: Go back to the previous screen\n"
	}, {
		"from": "gpt",
		"value": "<tool_call>\n{\"name\": \"computer_use\", \"arguments\": {\"action\": \"double_click\", \"coordinate\": [958, 61]}}\n</tool_call>"
	}],
	"image_id": "30/screenshot_0.png"
}
re_data = exec_box_from_gpt(data, resized_width_factor=2, resized_height_factor=2)
print(re_data)

{'image': '30/screenshot_0.png', 'conversations': [{'from': 'system', 'value': 'You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "mobile_use", "description": "Use a touchscreen to interact with a mobile device, and take screenshots.\\n* This is an interface to a mobile device with touchscreen. You can perform actions like clicking, typing, swiping, etc.\\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions.\\n* The screen\'s resolution is 952x2156.\\n* Make sure to click any buttons, links, icons, etc with the cursor tip in the center of the element. Don\'t click boxes on their edges unless asked.", "parameters": {"properties": {"action": {"description": "The action to perform. The available actions

In [6]:
def merge_consecutive_gpt_messages(data):
    conversations = data["conversations"]
    new_conversations = []
    buffer = []  # 临时存储连续的 gpt 消息
    
    for conv in conversations:
        if conv["from"] == "gpt":
            buffer.append(conv["value"])  # 缓存 gpt 消息
        else:
            # 如果 buffer 不为空，说明之前有 gpt 消息，需要合并
            if buffer:
                # 合并所有 gpt 消息，用 \n 连接
                merged_value = "\n".join(buffer)
                # 只保留最后一个 gpt 消息，并更新其 value
                new_conversations.append({"from": "gpt", "value": merged_value})
                buffer = []  # 清空 buffer
            new_conversations.append(conv)  # 添加非 gpt 消息
    
    # 处理末尾可能剩余的 gpt 消息
    if buffer:
        merged_value = "\n".join(buffer)
        new_conversations.append({"from": "gpt", "value": merged_value})
    
    data["conversations"] = new_conversations
    return data

data = {
  "image": "30/screenshot_0.png",
  "conversations": [
    {
      "from": "system",
      "value": "You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{\"type\": \"function\", \"function\": {\"name\": \"mobile_use\", \"description\": \"Use a touchscreen to interact with a mobile device, and take screenshots.\\n* This is an interface to a mobile device with touchscreen. You can perform actions like clicking, typing, swiping, etc.\\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions.\\n* The screen's resolution is 952x2156.\\n* Make sure to click any buttons, links, icons, etc with the cursor tip in the center of the element. Don't click boxes on their edges unless asked.\", \"parameters\": {\"properties\": {\"action\": {\"description\": \"The action to perform. The available actions are:\\n* `key`: Perform a key event on the mobile device.\\n    - This supports adb's `keyevent` syntax.\\n    - Examples: \\\"volume_up\\\", \\\"volume_down\\\", \\\"power\\\", \\\"camera\\\", \\\"clear\\\".\\n* `click`: Click the point on the screen with coordinate (x, y).\\n* `long_press`: Press the point on the screen with coordinate (x, y) for specified seconds.\\n* `swipe`: Swipe from the starting point with coordinate (x, y) to the end point with coordinates2 (x2, y2).\\n* `type`: Input the specified text into the activated input box.\\n* `system_button`: Press the system button.\\n* `open`: Open an app on the device.\\n* `wait`: Wait specified seconds for the change to happen.\\n* `terminate`: Terminate the current task and report its completion status.\", \"enum\": [\"key\", \"click\", \"long_press\", \"swipe\", \"type\", \"system_button\", \"open\", \"wait\", \"terminate\"], \"type\": \"string\"}, \"coordinate\": {\"description\": \"(x, y): The x (pixels from the left edge) and y (pixels from the top edge) coordinates to move the mouse to. Required only by `action=click`, `action=long_press`, and `action=swipe`.\", \"type\": \"array\"}, \"coordinate2\": {\"description\": \"(x, y): The x (pixels from the left edge) and y (pixels from the top edge) coordinates to move the mouse to. Required only by `action=swipe`.\", \"type\": \"array\"}, \"text\": {\"description\": \"Required only by `action=key`, `action=type`, and `action=open`.\", \"type\": \"string\"}, \"time\": {\"description\": \"The seconds to wait. Required only by `action=long_press` and `action=wait`.\", \"type\": \"number\"}, \"button\": {\"description\": \"Back means returning to the previous interface, Home means returning to the desktop, Menu means opening the application background menu, and Enter means pressing the enter. Required only by `action=system_button`\", \"enum\": [\"Back\", \"Home\", \"Menu\", \"Enter\"], \"type\": \"string\"}, \"status\": {\"description\": \"The status of the task. Required only by `action=terminate`.\", \"type\": \"string\", \"enum\": [\"success\", \"failure\"]}}, \"required\": [\"action\"], \"type\": \"object\"}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call>"
    },
    {
      "from": "system",
      "value": "You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{\"type\": \"function\", \"function\": {\"name\": \"mobile_use\", \"description\": \"Use a touchscreen to interact with a mobile device, and take screenshots.\\n* This is an interface to a mobile device with touchscreen. You can perform actions like clicking, typing, swiping, etc.\\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions.\\n* The screen's resolution is 952x2156.\\n* Make sure to click any buttons, links, icons, etc with the cursor tip in the center of the element. Don't click boxes on their edges unless asked.\", \"parameters\": {\"properties\": {\"action\": {\"description\": \"The action to perform. The available actions are:\\n* `key`: Perform a key event on the mobile device.\\n    - This supports adb's `keyevent` syntax.\\n    - Examples: \\\"volume_up\\\", \\\"volume_down\\\", \\\"power\\\", \\\"camera\\\", \\\"clear\\\".\\n* `click`: Click the point on the screen with coordinate (x, y).\\n* `long_press`: Press the point on the screen with coordinate (x, y) for specified seconds.\\n* `swipe`: Swipe from the starting point with coordinate (x, y) to the end point with coordinates2 (x2, y2).\\n* `type`: Input the specified text into the activated input box.\\n* `system_button`: Press the system button.\\n* `open`: Open an app on the device.\\n* `wait`: Wait specified seconds for the change to happen.\\n* `terminate`: Terminate the current task and report its completion status.\", \"enum\": [\"key\", \"click\", \"long_press\", \"swipe\", \"type\", \"system_button\", \"open\", \"wait\", \"terminate\"], \"type\": \"string\"}, \"coordinate\": {\"description\": \"(x, y): The x (pixels from the left edge) and y (pixels from the top edge) coordinates to move the mouse to. Required only by `action=click`, `action=long_press`, and `action=swipe`.\", \"type\": \"array\"}, \"coordinate2\": {\"description\": \"(x, y): The x (pixels from the left edge) and y (pixels from the top edge) coordinates to move the mouse to. Required only by `action=swipe`.\", \"type\": \"array\"}, \"text\": {\"description\": \"Required only by `action=key`, `action=type`, and `action=open`.\", \"type\": \"string\"}, \"time\": {\"description\": \"The seconds to wait. Required only by `action=long_press` and `action=wait`.\", \"type\": \"number\"}, \"button\": {\"description\": \"Back means returning to the previous interface, Home means returning to the desktop, Menu means opening the application background menu, and Enter means pressing the enter. Required only by `action=system_button`\", \"enum\": [\"Back\", \"Home\", \"Menu\", \"Enter\"], \"type\": \"string\"}, \"status\": {\"description\": \"The status of the task. Required only by `action=terminate`.\", \"type\": \"string\", \"enum\": [\"success\", \"failure\"]}}, \"required\": [\"action\"], \"type\": \"object\"}}}\n</tools>\n\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\n<tool_call>\n{\"name\": <function-name>, \"arguments\": <args-json-object>}\n</tool_call>"
    },
    {
      "from": "human",
      "value": "<image>\nPlease generate the next move according to the UI screenshot, instruction and previous actions.\n\nInstruction: Compare the prices of the Shoes similar to men's athletic shoes in the Etsy app\n\nPrevious actions:\nNone"
    },
    {
      "from": "human",
      "value": "<image>\nPlease generate the next move according to the UI screenshot, instruction and previous actions.\n\nInstruction: Compare the prices of the Shoes similar to men's athletic shoes in the Etsy app\n\nPrevious actions:\nNone"
    },
    {
      "from": "gpt",
      "value": "Action: Go back to the previous screen\n"
    },
    {
      "from": "gpt",
      "value": "<tool_call>\n{\"name\": \"mobile_use\", \"arguments\": {\"action\": \"system_button\", \"button\": \"Back\"}}\n</tool_call>"
    },
    {
      "from": "gpt",
      "value": "Action: Go back to the previous screen\n"
    },
    {
      "from": "gpt",
      "value": "<tool_call>\n{\"name\": \"mobile_use\", \"arguments\": {\"action\": \"system_button\", \"button\": \"Back\"}}\n</tool_call>"
    }
  ],
  "image_id": "30/screenshot_0.png"
}
new_data =  merge_consecutive_gpt_messages(data)
import json
#按照json格式打印
print(json.dumps(new_data, indent=4))

{
    "image": "30/screenshot_0.png",
    "conversations": [
        {
            "from": "system",
            "value": "You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{\"type\": \"function\", \"function\": {\"name\": \"mobile_use\", \"description\": \"Use a touchscreen to interact with a mobile device, and take screenshots.\\n* This is an interface to a mobile device with touchscreen. You can perform actions like clicking, typing, swiping, etc.\\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions.\\n* The screen's resolution is 952x2156.\\n* Make sure to click any buttons, links, icons, etc with the cursor tip in the center of the element. Don't click boxes on their edges unless asked.\", \"parameters\": {\"properties\": {\"action\": 

In [10]:
data_path_list = [
    '/braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/seeclick_mi_ui_tars_cleaned/seeclick_mi_ui_tars_cleaned.jsonl',
    '/braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/seeclick_ui_tars_cleaned_fixed/seeclick_ui_tars_cleaned_fixed.jsonl',
    '/braincoder-extreme-nas/datasets/Jedi_LF/datasets/refusal/refusal_seeclick_mi_ui_tars_cleaned/refusal_seeclick_mi_ui_tars_cleaned.jsonl',
    '/braincoder-extreme-nas/datasets/Jedi_LF/datasets/refusal/refusal_seeclick_ui_tars_cleaned_fixed/refusal_seeclick_ui_tars_cleaned_fixed.jsonl'
]
def chage_data_sharegpt(item):
    item['messages'] = []
    for conversation in item['conversations']:
        item['messages'].append({
                'content': conversation['value'],
                'role': conversation['from']
            })
    if 'conversations' in item:
        del item['conversations']
    if 'image' in item:
        del item['image']
    if 'image_id' in item:
        del item['image_id']
    return item


for data_path in data_path_list:
    data = []
    with open(data_path, 'r') as f:
        for line in f:
            tmp_data = json.loads(line)
            tmp_data = chage_data_sharegpt(tmp_data)
            if 'images' in tmp_data:
                img_path = tmp_data['images']
                img_list= []
                for img in img_path:
                    if 'Jedi_new' in img:
                        img = img.replace('Jedi_new', 'Jedi_LF')
                    img_list.append(img)
                tmp_data['images'] = img_list
            data.append(tmp_data)
    with open(data_path, 'w') as f:
        for item in data:
            f.write(json.dumps(item) + '\n')

In [91]:
import re

import re

def exec_box_from_human(item, factors=[]):
    new_conversations = []
    flag = -1
    
    for conversation in item['conversations']:
        if conversation['from'] == 'human' and '<image>' in conversation['value']:
            flag += 1
            
        if conversation['from'] == 'human':
            value = conversation['value']
            patterns = [
                # 1. 等号格式 (允许前缀)
                {
                    'regex': r'(\s*(?:bounding box|bbox|box|is)?\s*)(x\s*=\s*(\d+\.?\d*)\s*,\s*y\s*=\s*(\d+\.?\d*)\s*,\s*w\s*=\s*(\d+\.?\d*)\s*,\s*h\s*=\s*(\d+\.?\d*))',
                    'format': 'eq',
                    'prefix': 1,
                    'coords': [3, 4, 5, 6]
                },
                # 2. 标签格式 (x,y,w,h)
                {
                    'regex': r'(\s*(?:bounding box|bbox|box|is)?\s*)(x\s*,\s*y\s*,\s*w\s*,\s*h\s*:\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*))',
                    'format': 'label_xywh',
                    'prefix': 1,
                    'coords': [3, 4, 5, 6]
                },
                # 3. 标签格式 (w,h,x,y)
                {
                    'regex': r'(\s*(?:bounding box|bbox|box|is)?\s*)(w\s*,\s*h\s*,\s*x\s*,\s*y\s*:\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*))',
                    'format': 'label_whxy',
                    'prefix': 1,
                    'coords': [3, 4, 5, 6]
                },
                # 4. 括号格式 (捕获括号类型)
                {
                    'regex': r'(\s*(?:bounding box|bbox|box|is)?\s*)([\[\(]\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*[\]\)])',
                    'format': 'bracket',
                    'prefix': 1,
                    'coords': [3, 4, 5, 6]
                },
                # 5. 无标签四元组 (带前缀)
                {
                    'regex': r'(\s*(?:bounding box|bbox|box|is|:)\s*)((\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*))',
                    'format': 'plain',
                    'prefix': 1,
                    'coords': [3, 4, 5, 6]
                },
                # 6. 纯四元组 (无前缀)
                {
                    'regex': r'(\s*)((\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*)\s*,\s*(\d+\.?\d*))',
                    'format': 'plain',
                    'prefix': 1,
                    'coords': [3, 4, 5, 6]
                }
            ]
            
            found = False
            for pattern in patterns:
                match = re.search(pattern['regex'], value)
                if match:
                    try:
                        # 提取坐标值
                        prefix = match.group(pattern['prefix'])
                        coord_str = match.group(2)
                        
                        # 提取具体坐标值
                        coords = [float(match.group(i)) for i in pattern['coords']]
                        
                        # 检查缩放因子
                        if flag >= len(factors) or not factors[flag]:
                            scaled_coords = list(map(int, coords))
                        else:
                            w_factor, h_factor = factors[flag]
                            
                            # 根据格式调整缩放顺序
                            if pattern['format'] == 'label_whxy':
                                w, h, x, y = coords
                                scaled_coords = [
                                    int(w * w_factor),
                                    int(h * h_factor),
                                    int(x * w_factor),
                                    int(y * h_factor)
                                ]
                            else:
                                x, y, w, h = coords
                                scaled_coords = [
                                    int(x * w_factor),
                                    int(y * h_factor),
                                    int(w * w_factor),
                                    int(h * h_factor)
                                ]
                        
                        # 生成替换字符串
                        if pattern['format'] == 'eq':
                            replaced_coord = f"x={scaled_coords[0]}, y={scaled_coords[1]}, w={scaled_coords[2]}, h={scaled_coords[3]}"
                        elif pattern['format'] == 'label_xywh':
                            replaced_coord = f"x, y, w, h: {scaled_coords[0]}, {scaled_coords[1]}, {scaled_coords[2]}, {scaled_coords[3]}"
                        elif pattern['format'] == 'label_whxy':
                            replaced_coord = f"w, h, x, y: {scaled_coords[0]}, {scaled_coords[1]}, {scaled_coords[2]}, {scaled_coords[3]}"
                        elif pattern['format'] == 'bracket':
                            # 保留原始括号类型
                            bracket_open = '[' if '[' in coord_str else '('
                            bracket_close = ']' if '[' in coord_str else ')'
                            replaced_coord = f"{bracket_open}{scaled_coords[0]}, {scaled_coords[1]}, {scaled_coords[2]}, {scaled_coords[3]}{bracket_close}"
                        else:  # plain
                            replaced_coord = f"{scaled_coords[0]}, {scaled_coords[1]}, {scaled_coords[2]}, {scaled_coords[3]}"
                        
                        # 组合前缀和替换后的坐标
                        replaced = prefix + replaced_coord
                        
                        # 保留上下文替换
                        new_value = value.replace(match.group(0), replaced, 1)
                        new_conv = conversation.copy()
                        new_conv['value'] = new_value
                        new_conversations.append(new_conv)
                        found = True
                        break
                    
                    except Exception as e:
                        print(f"坐标处理错误: {e}")
                        continue  # 继续尝试其他模式
            
            if not found:
                new_conversations.append(conversation)
        else:
            new_conversations.append(conversation)
    
    item['conversations'] = new_conversations
    return item

data = {
	"image": "074ba6fb1b0013b749a5d9e6aea9ad4969d7fc4a91b37fbf85424119f301e060.png",
	"conversations": [{
		"from": "system",
		"value": "Operating as a GUI element descriptor, you will examine a screenshot and a bounding box (x, y, w, h values). Your responsibility is to provide an intuitive, detailed description of the UI component within the specified region, enabling users to locate it without coordinate references. Your description must address:\n\n1. Visual Makeup: Document the element's visual aspects, including its form, coloring, and design patterns\n2. Relative Position: Describe the element's placement in relation to surrounding interface elements\n3. Usage Pattern: Clarify the element's function and how users should interact with it\n4. Interface Element Type: Define the specific category of UI component (e.g., search bar, menu item)\n\nMaintain specificity and clarity, avoiding ambiguous descriptions. Focus on distinctive characteristics and spatial relationships that make the element readily identifiable."
	}, {
		"from": "human",
		"value": "<image>\nbounding box: x=353.5, y=862.3779, w=7.0, h=7.0868. Generate pls."
	}, {
		"from": "gpt",
		"value": "Let me describe this UI element:\n\nVisual Makeup: The element is a star icon with a solid orange color, representing a rating. It is part of a series of stars, typically used to indicate a product's rating. The star is visually distinct due to its bright color and geometric shape, which stands out against the white background of the product card.\n\nRelative Position: The star is located within the 'Best Seller' section of the 'Featured Products' area on the homepage. It is specifically part of the 'Red Capsicum' product card, positioned below the product name and price, and aligned horizontally with other stars in the rating row.\n\nUsage Pattern: The primary function of the star is to visually represent a rating for the product. Users can quickly assess the product's popularity or quality based on the number of filled stars. It serves as a quick visual cue for product evaluation.\n\nInterface Element Type: The element is a rating star, commonly used in e-commerce interfaces to display product ratings.",
		"recipient": "all",
		"end_turn": True
	}]
}




item = exec_box_from_human(data, factors=[[0.9, 0.9]])
import json
print(json.dumps(item, indent=4))

{
    "image": "074ba6fb1b0013b749a5d9e6aea9ad4969d7fc4a91b37fbf85424119f301e060.png",
    "conversations": [
        {
            "from": "system",
            "value": "Operating as a GUI element descriptor, you will examine a screenshot and a bounding box (x, y, w, h values). Your responsibility is to provide an intuitive, detailed description of the UI component within the specified region, enabling users to locate it without coordinate references. Your description must address:\n\n1. Visual Makeup: Document the element's visual aspects, including its form, coloring, and design patterns\n2. Relative Position: Describe the element's placement in relation to surrounding interface elements\n3. Usage Pattern: Clarify the element's function and how users should interact with it\n4. Interface Element Type: Define the specific category of UI component (e.g., search bar, menu item)\n\nMaintain specificity and clarity, avoiding ambiguous descriptions. Focus on distinctive characteristics a

In [4]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [10]:
import glob
import json
import os   
from tqdm import tqdm
def find_all_json_files(dir_path):
    """高效查找JSON文件，跳过images文件夹"""
    json_files = []
    for root, dirs, files in os.walk(dir_path):
        # 在遍历前就从dirs中移除images文件夹，避免进入该目录
        if 'images' in dirs:
            dirs.remove('images')  # 修改dirs列表会影响后续遍历
        
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

files = find_all_json_files('/braincoder-extreme-nas/datasets/Jedi_LF/')
total_line = 0
for file in files:
    if 'refusal' in file:
        continue
    data = json.load(open(file, 'r'))
    total_line += len(data)
print(total_line)
# files = ['/braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/seeclick_mi_ui_tars_cleaned/seeclick_mi_ui_tars_cleaned.json']
# print(len(files))
# for file in files:
#     data = json.load(open(file, 'r'))
#     for item in tqdm(data):
#         assert 'images' in item, f"images not in {file}"
#         for image in item['images']:
#             tmp_img = image.replace('primus_datasets', 'braincoder-extreme-nas')
#             assert os.path.exists(tmp_img), f"image not exist: {tmp_img}"
            

3741994


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [1]:
[
  {
    "images": [
      "/primus_datasets/datasets/Jedi_LF/datasets/aguvis++/android_control-v2/images/512/screenshot_2.png"
    ],
    "messages": [
      {
        "content": "",
        "role": "system"
      },
      {
        "content": "",
        "role": "user"
      },
      {
        "content": "",
        "role": "assistant"
      }
    ]
  },
]

[{'images': ['/primus_datasets/datasets/Jedi_LF/datasets/aguvis++/android_control-v2/images/512/screenshot_2.png'],
  'messages': [{'content': '', 'role': 'system'},
   {'content': '', 'role': 'user'},
   {'content': '', 'role': 'assistant'}]}]

In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [8]:
import json
data = json.load(open('/braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/android_control-v2/android_control-v2.json', 'r'))
print(len(data))
print(data[0])

54678
{'images': ['/primus_datasets/datasets/Jedi_LF/datasets/aguvis++/android_control-v2/images/512/screenshot_2.png'], 'messages': [{'content': 'You are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "mobile_use", "description": "Use a touchscreen to interact with a mobile device, and take screenshots.\\n* This is an interface to a mobile device with touchscreen. You can perform actions like clicking, typing, swiping, etc.\\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions.\\n* The screen\'s resolution is 1008x2044.\\n* Make sure to click any buttons, links, icons, etc with the cursor tip in the center of the element. Don\'t click boxes on their edges unless asked.", "parameters": {"properties": {"ac

In [ ]:
import json
data = json.load(open('/home/admin/caichenglin/code/llama-factory/data/dataset_info.json', 'r'))
keys = []
no_r_key = []
for key, value in data.items():
    if '_jedi' in key:
        keys.append(key)
        if 'refusal' not in key:
            no_r_key.append(key)
print(', '.join(no_r_key))
print(len(no_r_key))
print(len(keys))

seeclick_mi_ui_tars_cleaned_jedi, seeclick_ui_tars_cleaned_fixed_jedi, android_control-v2_jedi, coat-v2_jedi, guiact-web-multi_jedi, guiact-web-single-v2_jedi, guide_si_10k-v2_jedi, guienv_jedi, mind2web_train_v1.0.1_jedi, omniact_jedi, ricoig16k_jedi, ricosca_jedi, ui_refexp_jedi, webui350k_jedi, widget_captioning_jedi, component_final_1.5m_cleaned_split_jedi, component_library_snap_icon_data_description_jedi, component_library_snap_icon_data_grounding_jedi, component_v1_130k_jedi, doc_data_new_jedi, doc_scroll_data_new_jedi, ethercalc_v1_jedi, slide_v1_17k_jedi, icon_v0222_description_jedi, ios_app_data_jedi, mac_app_data_jedi, training_data_icon_jedi, icon_v0222_grounding_jedi, training_data_icon_grounded_merged_jedi, layout200k_jedi, layout200k_grounding_jedi, layout400k_claude_jedi, layout400k_claude_grounding_jedi, os_layout_v1_jedi, os_layout_v1_grounding_jedi
35
44


In [29]:
import json
data_path = '/braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/seeclick_mi_ui_tars_cleaned/seeclick_mi_ui_tars_cleaned.json'
data = json.load(open(data_path, 'r'))
new_data = []
for item in data[:200]:
    imgs = item['images']
    new_imgs = []
    for img in imgs:
        new_imgs.append(img.replace('primus_datasets', 'braincoder-extreme-nas'))
    item['images'] = new_imgs
    new_data.append(item)
new_data_path = data_path.replace('seeclick_mi_ui_tars_cleaned.json', 'seeclick_mi_ui_tars_cleaned_local.json')
json.dump(new_data, open(new_data_path, 'w'), ensure_ascii=False, indent=2)



In [12]:
import json
data_path = '/home/admin/caichenglin/code/llama-factory/gen.jsonl'
data = [json.loads(line) for line in open(data_path, 'r')]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
print(json.loads(data[0])

{'prompt': 'system\nYou are a helpful assistant.\n\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name_for_human": "computer_use", "name": "computer_use", "description": "Use a mouse and keyboard to interact with a computer, and take screenshots.\n* This is an interface to a desktop GUI. You do not have access to a terminal or applications menu. You must click on desktop icons to start applications.\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions. E.g. if you click on Firefox and a window doesn\'t open, try wait and taking another screenshot.\n* The screen\'s resolution is 1092x2408.\n* Whenever you intend to move the cursor to click on an element like an icon, you should consult a screenshot to determine the coordinates of the

In [9]:
import json
# print(json.dumps(data[0], indent=2))
# print(data[0]['predict'].replace('<tool_call>', '').replace('</tool_call>', ''))
data_path = '/home/admin/caichenglin/code/llama-factory/generated_predictions.jsonl'
data = [json.loads(line) for line in open(data_path, 'r')]
def extract_actions(text):
    """从文本中提取所有动作信息"""
    actions = []
    # 分割多个tool_call
    parts = text.split('</tool_call>')
    for part in parts:
        if '<tool_call>' not in part:
            continue
        # 提取JSON部分
        json_str = part.split('<tool_call>')[1].strip()
        try:
            data = json.loads(json_str)
            if data['name'] == 'computer_use':
                action_data = {
                    'action': data['arguments']['action'],
                    'coordinate': data['arguments'].get('coordinate', None)
                }
                actions.append(action_data)
        except json.JSONDecodeError as e:
            print(f"JSON解析错误: {e}")
    print(actions)
    return actions

def compare_actions(predict_actions, label_actions):
    """比较预测动作和标签动作"""
    comparisons = []
    for i, (pred, lbl) in enumerate(zip(predict_actions, label_actions), 1):
        comparison = {
            'action_number': i,
            'action_type_match': pred['action'] == lbl['action'],
            'pred_action': pred['action'],
            'label_action': lbl['action'],
            'coordinate_comparison': None
        }
        
        if pred['coordinate'] and lbl['coordinate']:
            comparison['coordinate_comparison'] = {
                'pred_coordinate': pred['coordinate'],
                'label_coordinate': lbl['coordinate'],
                'x_diff': abs(pred['coordinate'][0] - lbl['coordinate'][0]),
                'y_diff': abs(pred['coordinate'][1] - lbl['coordinate'][1]),
                'total_diff': ((pred['coordinate'][0] - lbl['coordinate'][0])**2 + 
                              (pred['coordinate'][1] - lbl['coordinate'][1])**2)**0.5
            }
        
        comparisons.append(comparison)
    return comparisons
exact_match = 0
soft_match_10 = 0
soft_match_20 = 0
action_match = 0
total_action = 0
total_coordinate = 0
diff_gap_10 = 10
diff_gap_20 = 20

for tmp_data in data[:]:
    predict_actions = extract_actions(tmp_data['predict'])
    label_actions = extract_actions(tmp_data['label'])
    results = compare_actions(predict_actions, label_actions)
    for item in results:
        total_action += 1
        if item['action_type_match']:
            action_match += 1
        if item['coordinate_comparison'] is not None:
            total_coordinate += 1
            if item['coordinate_comparison']['x_diff'] == 0 and item['coordinate_comparison']['y_diff'] == 0:
                exact_match += 1
            if item['coordinate_comparison']['x_diff'] <= diff_gap_10 and item['coordinate_comparison']['y_diff'] <= diff_gap_10:
                soft_match_10 += 1
            if item['coordinate_comparison']['x_diff'] <= diff_gap_20 and item['coordinate_comparison']['y_diff'] <= diff_gap_20:
                soft_match_20 += 1
print(total_coordinate, exact_match, soft_match_10, soft_match_20, action_match, total_action)
print(f'exact_match_rate: {exact_match/total_coordinate}, soft_match_rate_10: {soft_match_10/total_coordinate}, soft_match_rate_20: {soft_match_20/total_coordinate}, action_match_rate: {action_match/total_action}, total_action: {total_action}, total_coordinate: {total_coordinate}')
    
        

[{'action': 'left_click', 'coordinate': [987, 47]}]
[]


KeyboardInterrupt: 

In [22]:
# 读取/braincoder-extreme-nas/datasets/Jedi/datasets.yaml，统计每个json路径下json的行数,组织一个map，key是json的文件名称（去掉.json），value是行数
import yaml
import os
from tabulate import tabulate
from tqdm import tqdm

def count_jsonl_lines(file_path):
    """统计JSONL文件的行数"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return len(f.readlines())
    except FileNotFoundError:
        return "文件不存在"
    except Exception as e:
        return f"读取错误: {str(e)}"

def generate_markdown_table(yaml_path):
    """生成Markdown格式的统计表格"""
    with open(yaml_path, 'r', encoding='utf-8') as f:
        data = yaml.safe_load(f)
    
    table_data = []
    
    if 'datasets' in data:
        for dataset in tqdm(data['datasets']):
            if 'jsonl_path' in dataset:
                jsonl_path = os.path.join('/braincoder-extreme-nas/datasets/Jedi', dataset['jsonl_path'])
                if jsonl_path.endswith('.jsonl'):
                    base_name = os.path.basename(jsonl_path).replace('.jsonl', '')
                    line_count = count_jsonl_lines(jsonl_path)
                    table_data.append([base_name, line_count])
    
    # 使用tabulate生成Markdown表格
    headers = ["数据集名称", "行数"]
    markdown_table = tabulate(table_data, headers, tablefmt="github")
    
    # 添加标题和格式装饰
    result = f"""# JSONL数据集行数统计\n\n"""
    result += f"**统计时间**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"
    result += markdown_table
    
    return result

# 使用示例
from datetime import datetime
yaml_file_path = '/braincoder-extreme-nas/datasets/Jedi/datasets.yaml'
markdown_result = generate_markdown_table(yaml_file_path)

# 打印结果
print(markdown_result)


  0%|          | 0/47 [00:00<?, ?it/s]

100%|██████████| 47/47 [03:32<00:00,  4.52s/it]

# JSONL数据集行数统计

**统计时间**: 2025-06-17 09:00:54

| 数据集名称                                                       |    行数 |
|------------------------------------------------------------------|---------|
| seeclick_ui_tars_cleaned_fixed                                   |   69634 |
| guienv                                                           |  327972 |
| webui350k                                                        |   57389 |
| omniact                                                          |    6720 |
| seeclick_mi_ui_tars_cleaned                                      |   89999 |
| osatlas_ui_tars_cleaned                                          |  303472 |
| widget_captioning                                                |  101426 |
| ricosca                                                          |  173212 |
| ui_refexp                                                        |   15624 |
| docvqa_grounding                                                 |   34060 |
| ricoig16k 

In [27]:
# 
data_path = '/home/admin/caichenglin/code/llama-factory/data/dataset_info.json'
data_info = json.load(open(data_path, 'r'))

dataset_name_map = {}
table_data = []
for dataset_name, dataset_info in data_info.items():
    if not dataset_name.endswith('jedi'):
        continue
    
    file_name = dataset_info['file_name'].replace('primus_datasets', 'braincoder-extreme-nas')
    print(dataset_name, '   ',file_name)
    try:
        table_data.append([dataset_name,len(json.load(open(file_name, 'r')))])
    except Exception as e:
        print(e, file_name)
        continue

# 使用tabulate生成Markdown表格
headers = ["数据集名称", "行数"]
markdown_table = tabulate(table_data, headers, tablefmt="github")
print(markdown_table)


seeclick_mi_ui_tars_cleaned_jedi     /braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/seeclick_mi_ui_tars_cleaned/seeclick_mi_ui_tars_cleaned.json
seeclick_ui_tars_cleaned_fixed_jedi     /braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/seeclick_ui_tars_cleaned_fixed/seeclick_ui_tars_cleaned_fixed.json
android_control-v2_jedi     /braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/android_control-v2/android_control-v2.json
Expecting ',' delimiter: line 213440 column 23 (char 47848477) /braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/android_control-v2/android_control-v2.json
coat-v2_jedi     /braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/coat-v2/coat-v2.json
guiact-web-multi_jedi     /braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/guiact-web-multi/guiact-web-multi.json
guiact-web-single-v2_jedi     /braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/guiact-web-single-v2/guiact-web-single-v2.json
guide_si_10k-v2_jedi     

In [1]:
import os
from PIL import Image

path = "/braincoder-extreme-nas/datasets/aguvis-stage1/data/aguvis/images/seeclick/seeclick_web_imgs"
count = 0

for root, dirs, files in os.walk(path):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')):
            try:
                with Image.open(os.path.join(root, file)) as img:
                    print(f"{file}: {img.size[0]} x {img.size[1]}")
                    count += 1
                    if count >= 100: exit()
            except Exception as e:
                print(f"Error with {file}: {str(e)}")

KeyboardInterrupt: 

In [2]:
from datasets import Dataset, load_dataset
import json
import os
import random

def json_to_parquet_shards(json_path, output_dir, num_shards):
    # 1. 加载JSON数据
    with open(json_path, 'r') as f:
        data = json.load(f)  # 假设是列表格式的JSON
    
    # 2. 创建Dataset对象
    dataset = Dataset.from_list(data).shuffle(seed=42)
    
    # 3. 创建输出目录
    os.makedirs(output_dir, exist_ok=True)
    
    # 4. 保存为分片Parquet (自动处理)
    dataset.save_to_disk(
        dataset_path=output_dir,
        num_shards=num_shards,
        num_proc=8  # 并行进程数
    )
    print(f"数据已保存为 {num_shards} 个Parquet分片到 {output_dir}")

# 使用示例
json_to_parquet_shards(
    json_path="/braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/android_control-v2/android_control-v2.json",
    output_dir="/braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/android_control-v2/parquet_shards",
    num_shards=8
)

# 使用示例





Saving the dataset (8/8 shards): 100%|██████████| 54678/54678 [00:09<00:00, 5854.53 examples/s]


数据已保存为 8 个Parquet分片到 /braincoder-extreme-nas/datasets/Jedi_LF/datasets/aguvis++/android_control-v2/parquet_shards


cp /primus_datasets/datasets/Jedi_LF/datasets/aguvis++/android_control-v2/parquet_shards  /root/code/llama-factory/data/parquet_shards

export should_log=Ture 
cd /root/code/llama-factory 
FORCE_TORCHRUN=1 NNODES=$NNODES NODE_RANK=$RANK MASTER_ADDR=$MASTER_ADDR MASTER_PORT=$MASTER_PORT llamafactory-cli train examples/train_full/qwen2_5vl_full_sft_jedi_without_speedup_no-stream.yaml

In [4]:
# 读取json数据，91划分训练测试
data_path = '/braincoder-extreme-nas/datasets/Jedi_LF_new/datasets/aguvis++/android_control-v2/android_control-v2.json'
import json

data = json.load(open(data_path, 'r'))

for item in data:
    images = []
    for image in item['images']:
        images.append(image.replace('Jedi_LF_new', 'Jedi_LF'))
    item['images'] = images

        

train_data = data[:int(len(data)*0.9)]
test_data = data[int(len(data)*0.9):]


json.dump(data, open(data_path, 'w'), indent=4, ensure_ascii=False)

json.dump(train_data, open('/braincoder-extreme-nas/datasets/Jedi_LF_new/datasets/aguvis++/android_control-v2/android_control-v2_train.json', 'w'), indent=4, ensure_ascii=False)
json.dump(test_data, open('/braincoder-extreme-nas/datasets/Jedi_LF_new/datasets/aguvis++/android_control-v2/android_control-v2_test.json', 'w'), indent=4, ensure_ascii=False)


In [3]:
data_path = '/braincoder-extreme-nas/datasets/Jedi_LF_new/mixture_data/diff_img_size/diff_img_size_data_total_122248.json'
import json

data = json.load(open(data_path, 'r'))

for item in data:
    images = []
    for image in item['images']:
        images.append(image.replace('Jedi_LF_new', 'Jedi_LF'))
    item['images'] = images

output_file_path = '/braincoder-extreme-nas/datasets/Jedi_LF_new/mixture_data/diff_img_size/diff_img_size_data_total_122248_new.json'
json.dump(data, open(output_file_path, 'w'), indent=4, ensure_ascii=False)

llamafactory-cli train   --model_name_or_path /primus_datasets_xpfs/models/Qwen2.5-VL-7B-Instruct  --image_max_pixels 3211264  --video_max_pixels 16384  --trust_remote_code  --stage sft --do_train --finetuning_type full --freeze_vision_tower --freeze_multi_modal_projector --freeze_language_model false --deepspeed examples/deepspeed/ds_z2_config.json --dataset mix_img_size_jedi --template qwen2_vl --cutoff_len 16384 --overwrite_cache --preprocessing_num_workers 32 --dataloader_num_workers 16 --output_dir /primus_oss/save_checkpoint/  --logging_steps 10  --save_steps 2000  --plot_loss  --overwrite_output_dir --save_only_model false  --report_to none   --per_device_train_batch_size 2  --gradient_accumulation_steps 2  --learning_rate 1.0e-5   --num_train_epochs 3.0  --lr_scheduler_type cosine  --warmup_ratio 0.1  --bf16 --ddp_timeout 180000000  --flash_attn fa2  --enable_liger_kernel

diff_img_size_data_total_122248_new.json

In [2]:
data_path = '/home/admin/caichenglin/code/ScreenSpot-Pro-GUI-Grounding/screenspot_v1_data/pro_format_data/screenspot_desktop_convert.json'
output_path = '/home/admin/caichenglin/code/ScreenSpot-Pro-GUI-Grounding/screenspot_v1_data/pro_format_data/screenspot_desktop_convert_10.json'
import json
import random

data = json.load(open(data_path, 'r'))
random.shuffle(data)
sample_data = data[:10]

with open(output_path, 'w') as f:
    json.dump(sample_data, f, indent=4, ensure_ascii=False)


In [11]:
lf_path = '/home/admin/caichenglin/code/ScreenSpot-Pro-GUI-Grounding/results/img_size_checkpoint-2000/vll_batch_outputs_1.pkl'
ss_path = '/home/admin/caichenglin/code/ScreenSpot-Pro-GUI-Grounding/results/img_size_checkpoint-2000/batch_outputs.pkl'
import pickle as pkl

with open(lf_path, 'rb') as f:
    lf_outputs = pkl.load(f)

with open(ss_path, 'rb') as f:
    ss_outputs = pkl.load(f)

print(lf_outputs[0])
print(ss_outputs[0])






RequestOutput(request_id=0, prompt='<|im_start|>system\nYou are a helpful assistant.\n\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name_for_human": "computer_use", "name": "computer_use", "description": "Use a mouse and keyboard to interact with a computer, and take screenshots.\n* This is an interface to a desktop GUI. You do not have access to a terminal or applications menu. You must click on desktop icons to start applications.\n* Some applications may take time to start or process actions, so you may need to wait and take successive screenshots to see the results of your actions. E.g. if you click on Firefox and a window doesn\'t open, try wait and taking another screenshot.\n* The screen\'s resolution is 1092x2408.\n* Whenever you intend to move the cursor to click on an element like an icon, you should consult a screenshot